In [68]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [69]:
ds=pd.read_csv('daily_MSFT.csv',index_col="timestamp",parse_dates=True)

In [70]:
ds.head()

,open,high,low,close,volume
timestamp,,,,,
2019-01-02,99.55,101.75,98.94,101.12,35329345
2018-12-31,101.29,102.40,100.44,101.57,33173765
2018-12-28,102.09,102.41,99.52,100.39,38169312
2018-12-27,99.30,101.19,96.40,101.18,49498509
2018-12-26,95.14,100.69,93.96,100.56,51634793


In [71]:
ds.isna().any()

open      False
high      False
low       False
close     False
volume    False
dtype: bool

In [72]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5284 entries, 2019-01-02 to 1998-01-02
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    5284 non-null   float64
 1   high    5284 non-null   float64
 2   low     5284 non-null   float64
 3   close   5284 non-null   float64
 4   volume  5284 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 247.7 KB


In [73]:
ds['open'].plot(figsize=(16,10))
plt.show()

In [74]:
ds["close"]=ds['close'].astype(str).str.replace(',','').astype(float)

In [75]:
ds["volume"]=ds['volume'].astype(str).str.replace(',','').astype(float)

In [76]:
ds.head()

,open,high,low,close,volume
timestamp,,,,,
2019-01-02,99.55,101.75,98.94,101.12,35329345.0
2018-12-31,101.29,102.40,100.44,101.57,33173765.0
2018-12-28,102.09,102.41,99.52,100.39,38169312.0
2018-12-27,99.30,101.19,96.40,101.18,49498509.0
2018-12-26,95.14,100.69,93.96,100.56,51634793.0


In [77]:
ds.rolling(7).mean().head(20)

,open,high,low,close,volume
timestamp,,,,,
2019-01-02,NaN,NaN,NaN,NaN,NaN
2018-12-31,NaN,NaN,NaN,NaN,NaN
2018-12-28,NaN,NaN,NaN,NaN,NaN
2018-12-27,NaN,NaN,NaN,NaN,NaN
2018-12-26,NaN,NaN,NaN,NaN,NaN
2018-12-24,NaN,NaN,NaN,NaN,NaN
2018-12-21,99.525714,101.344286,97.242857,99.597143,5.185471e+07
2018-12-20,100.025714,101.710000,97.220000,99.652857,5.685540e+07
2018-12-19,100.362857,102.350000,97.350000,99.955714,6.185889e+07


In [78]:
ds['open'].plot(figsize=(16,6))
ds.rolling(window=30).mean()['close'].plot()
pylab.show()

In [79]:
ds['close: 30 day mean']=ds['close'].rolling(window=30).mean()
ds[['close','close: 30 day mean']].plot(figsize=(16,6))

<AxesSubplot:xlabel='timestamp'>

In [80]:
ds['close'].expanding(min_periods=1).mean().plot(figsize=(16,6))

<AxesSubplot:xlabel='timestamp'>

In [81]:
training_set=ds['open']
training_set=pd.DataFrame(training_set)

In [82]:
#feature scaling

from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler(feature_range=(0,1))
training_set_scaled=sc.fit_transform(training_set)

In [83]:
x_train=[]
y_train=[]
for i in range(60,1258):
    x_train.append(training_set_scaled[i-60:i,0])
    y_train.append(training_set_scaled[i,0])
x_train,y_train =np.array(x_train),np.array(y_train)

x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))

In [84]:
#RNN

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [85]:
regressor=Sequential()

In [86]:
regressor.add(LSTM(units=50,return_sequences=True,input_shape=(x_train.shape[1],1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50,return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50,return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units=1))

In [87]:
regressor.compile(optimizer='adam',loss='mean_squared_error')

regressor.fit(x_train,y_train,epochs=100,batch_size=32)

Epoch 1/100
38/38 [==============================] - 12s 115ms/step - loss: 0.0121
Epoch 2/100
38/38 [==============================] - 3s 79ms/step - loss: 0.0013
Epoch 3/100
38/38 [==============================] - 3s 79ms/step - loss: 0.0013
Epoch 4/100
38/38 [==============================] - 3s 86ms/step - loss: 0.0012
Epoch 5/100
38/38 [==============================] - 4s 118ms/step - loss: 0.0010ETA: 1s - loss: 9.6 - ETA: 0s - loss: 
Epoch 6/100
38/38 [==============================] - 4s 104ms/step - loss: 9.8287e-04
Epoch 7/100
38/38 [==============================] - 4s 94ms/step - loss: 8.1240e-04
Epoch 8/100
38/38 [==============================] - 3s 83ms/step - loss: 9.3470e-04
Epoch 9/100
38/38 [==============================] - 4s 101ms/step - loss: 8.8783e-04
Epoch 10/100
38/38 [==============================] - 4s 94ms/step - loss: 8.5092e-04
Epoch 11/100
38/38 [==============================] - 3s 82ms/step - loss: 0.0010
Epoch 12/100
38/38 [========================

38/38 [==============================] - 3s 76ms/step - loss: 2.8011e-04
Epoch 97/100
38/38 [==============================] - 3s 77ms/step - loss: 2.9814e-04
Epoch 98/100
38/38 [==============================] - 3s 76ms/step - loss: 3.2183e-04
Epoch 99/100
38/38 [==============================] - 3s 79ms/step - loss: 2.7895e-04
Epoch 100/100
38/38 [==============================] - 3s 88ms/step - loss: 2.8709e-04


In [96]:
stock_price=ds.iloc[:1,:2].values
ds.head()

,open,high,low,close,volume,close: 30 day mean
timestamp,,,,,,
2019-01-02,99.55,101.75,98.94,101.12,35329345.0,NaN
2018-12-31,101.29,102.40,100.44,101.57,33173765.0,NaN
2018-12-28,102.09,102.41,99.52,100.39,38169312.0,NaN
2018-12-27,99.30,101.19,96.40,101.18,49498509.0,NaN
2018-12-26,95.14,100.69,93.96,100.56,51634793.0,NaN


In [97]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5284 entries, 2019-01-02 to 1998-01-02
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   open                5284 non-null   float64
 1   high                5284 non-null   float64
 2   low                 5284 non-null   float64
 3   close               5284 non-null   float64
 4   volume              5284 non-null   float64
 5   close: 30 day mean  5255 non-null   float64
dtypes: float64(6)
memory usage: 289.0 KB


In [98]:
ds['volume']=ds['volume'].astype(str).str.replace(',','').astype(float)

In [91]:
test_set=ds['open']
test_set=pd.DataFrame(test_set)

In [92]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5284 entries, 2019-01-02 to 1998-01-02
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    5284 non-null   float64
dtypes: float64(1)
memory usage: 82.6 KB


In [93]:
dataset_total=pd.concat((ds['open'],ds['open']),axis=0)
inputs=dataset_total[len(dataset_total)-len(ds)-60:].values
inputs=inputs.reshape(-1,1)
inputs=sc.transform(inputs)
x_test=[]
for i in range(60,80):
    x_test.append(inputs[i-60:i,0])
x_test=np.array(x_test)
x_test=np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
predicted_stock_price=regressor.predict(x_test)
predicted_stock_proce=sc.inverse_transform(predicted_stock_price)

In [94]:
predicted_stock_price=pd.DataFrame(predicted_stock_price)
predicted_stock_price.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       20 non-null     float32
dtypes: float32(1)
memory usage: 208.0 bytes


In [100]:
plt.plot(stock_price,color='red',label='Real Microsoft Stock Price')
plt.plot(predicted_stock_price,color='blue',label='Predicted Microsoft Stock Price')
plt.title('Microsoft Stock Price Predictor')
plt.xlabel('time')
plt.ylabel('Microsoft Stock price')
plt.legend()
plt.show()